## Loading Data and Initial Analysis

Importing all packages

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

Using SQLite and importing the dataset using the db file. Displaying the tables

In [5]:
con = sqlite3.connect('./archive/database.sqlite')

pd.read_sql_query("""SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;""", con)

,name
0,Reviews


Displaying the data

In [6]:
pd.read_sql_query("SELECT * FROM Reviews", con)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


Importing only 10000 values cuz bad pc. I'm ignoring the reviews with 3 rating as they are mostly neutral and does not offer much. 

In [15]:
data = pd.read_sql_query("SELECT * FROM Reviews WHERE Score != 3 LIMIT 10000 ", con)
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


If the rating is 1 or 2, giving it a value of 0( Negative ), and if the rating is 4 or 5, giving it a value of 1( Positive ).

In [16]:
def mod_score(x):
    if x < 3:
        return 0
    else:
        return 1
    
original_score = data['Score']
mod_score = original_score.map(mod_score)
data['Score'] = mod_score
print(f"Size of dataset: {data.shape}")
data.head()

Size of dataset: (10000, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,Great taffy at a great price. There was a wid...


Seeing number of people that submitted more than 1 review

In [23]:
mul_reviews = pd.read_sql_query("""
                              SELECT UserId, ProfileName, ProductId, Time, Score, Text, COUNT(*) 
                              FROM Reviews
                              GROUP BY UserId
                              HAVING COUNT(*) > 1
                              """, con)
print(f"Number of users that have submitted more than 1 reviews are: {mul_reviews.shape[0]}")
print(f"Total number of reviews posted by these people is {mul_reviews['COUNT(*)'].sum()}")
mul_reviews.head()

Number of users that have submitted more than 1 reviews are: 80668
Total number of reviews posted by these people is 393063


,UserId,ProfileName,ProductId,Time,Score,Text,COUNT(*)
0,#oc-R115TNMSPFT9I7,Breyton,B005ZBZLT4,1331510400,2,Overall its just OK when considering the price...,2
1,#oc-R11D9D7SHXIJB9,"Louis E. Emory ""hoppy""",B005HG9ESG,1342396800,5,"My wife has recurring extreme muscle spasms, u...",3
2,#oc-R11DNU2NBKQ23Z,Kim Cieszykowski,B005ZBZLT4,1348531200,1,This coffee is horrible and unfortunately not ...,2
3,#oc-R11O5J5ZVQE25C,Penguin Chick,B005HG9ESG,1346889600,5,This will be the bottle that you grab from the...,3
4,#oc-R12KPBODL2B5ZD,Christopher P. Presta,B007OSBEV0,1348617600,1,I didnt like this coffee. Instead of telling y...,2


## Exploratory Data Analysis
### Data Cleaning: Removing Duplicates/Deduplication

In [24]:
sorted_data = data.sort_values('ProductId', axis = 0, ascending = True, inplace = False, kind = 'quicksort', na_position = 'last')
print(f"Shape after sorting, including duplicate values: {sorted_data.shape}")
final = sorted_data.drop_duplicates(subset={"UserId", "ProfileName", "Time", "Text"}, keep = 'first', inplace = False)
print(f"Shape after sorting and removing duplicate values: {final.shape}")

Shape after sorting, including duplicate values: (10000, 10)
Shape after sorting and removing duplicate values: (9564, 10)


In [26]:
print(f"Percent of data retained after dropping duplicate elements: {((final['Id'].size * 1.0) / (data['Id'].size*1.0) * 100)}")

Percent of data retained after dropping duplicate elements: 95.64


Some records have helpful numerator > helpful denominator. This should not be possible and is probably an error in the dataset. These need to filtered out. 

Helpful Numerator = Number of people that found the review helpful
Helpful Denominator = Total Number of people that found the review helpful and not helpful

In [27]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [29]:
final['Score'].value_counts()

1    7976
0    1588
Name: Score, dtype: int64